In [23]:
#@title 8.7.1. Import some necessary packages
import numpy as np
import pandas as pd
import glob
import time

from sklearn.model_selection import train_test_split

import matplotlib
from matplotlib import pyplot as plt

import tensorflow as tf
import keras
from keras import layers, models

# Search online about "ADAM" parameters
opt = keras.optimizers.Adam(learning_rate=0.00005, beta_1=0.5)

from IPython.display import clear_output

In [24]:
#@title 8.7.3. Discriminator function
def fun_discriminator():
  model = models.Sequential()

  # Search online to learn about "LeakyReLU"

  model.add(layers.Conv2D(256, (3,3), strides=(2, 2), padding='same', input_shape=(96,96,1) ))
  model.add(layers.LeakyReLU(alpha=0.2))
  model.add(layers.Dropout(0.4))

  model.add(layers.Conv2D(256, (3,3), strides=(2, 2), padding='same'))
  model.add(layers.LeakyReLU(alpha=0.2))
  model.add(layers.Dropout(0.4))

  model.add(layers.Flatten())
  model.add(layers.Dense(1, activation='sigmoid'))

  # Search online about "ADAM" parameters

  model.compile(loss='mean_squared_error', optimizer=opt)

  return model

In [25]:
#@title 8.7.4. Generator function
def fun_generator(hidden_size):
  model = models.Sequential()

  model.add(layers.Dense(128 * 16* 16, input_dim=hidden_size))
  model.add(layers.LeakyReLU(alpha=0.2))
  model.add(layers.Reshape((16, 16, 128)))

  model.add(layers.Conv2DTranspose(512, (3,3), strides=(2,2), padding='same'))
  model.add(layers.LeakyReLU(alpha=0.2))

  model.add(layers.Conv2DTranspose(512, (3,3), strides=(2,2), padding='same'))
  model.add(layers.LeakyReLU(alpha=0.2))

  model.add(layers.Conv2DTranspose(512, (3,3), strides=(2,2), padding='same'))
  model.add(layers.LeakyReLU(alpha=0.2))

  model.add(layers.Conv2D(1, (33,33), strides=(1,1), activation='sigmoid'))

  #model.summary()

  # We do not compile it. We need to add discriminator to the generator then compile it

  return model

In [26]:
#@title 8.7.5. Gan function
def fun_gan(model_discriminator, model_generator):

  # We just want the generator parameter being updated each time we call model_gan
  model_discriminator.trainable = False

  model = models.Sequential()

  model.add(model_generator)
  model.add(model_discriminator)

  model.compile(loss='mean_squared_error', optimizer=opt)

  return model

In [28]:
#@title 8.7.7. Training parameters and GAN models
batch_size  = 8
hidden_size = 32
epochs      = 50

num_batch   = int(datain.shape[0]/batch_size)

model_discriminator = fun_discriminator()
model_generator     = fun_generator(hidden_size)
model_gan           = fun_gan(model_discriminator, model_generator)

print("$$$$$$$$$$$$$$$$$$$$$$$$$")
print('Discriminator')
print("$$$$$$$$$$$$$$$$$$$$$$$$$")
model_discriminator.summary()

print("$$$$$$$$$$$$$$$$$$$$$$$$$")
print('Generator')
print("$$$$$$$$$$$$$$$$$$$$$$$$$")
model_generator.summary()

print("$$$$$$$$$$$$$$$$$$$$$$$$$")
print('GAN')
print("$$$$$$$$$$$$$$$$$$$$$$$$$")
model_gan.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


$$$$$$$$$$$$$$$$$$$$$$$$$
Discriminator
$$$$$$$$$$$$$$$$$$$$$$$$$


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 48, 48, 256)         │           2,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 48, 48, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 48, 48, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 24, 24, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 24, 24, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 24, 24, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 147456)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │         147,457 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 740,097 (2.82 MB)

...

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_1 (Sequential)            │ (None, 96, 96, 1)           │       6,948,865 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 1)                   │         740,097 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,688,962 (29.33 MB)

 Trainable params: 6,948,865 (26.51 MB)

 Non-trainable params: 740,097 (2.82 MB)

In [ ]:
#@title 8.7.8. Training
for i0 in range(epochs):
  for i1 in range(num_batch):
    hiddenin,_               = fun_latent(batch_size, hidden_size)

    datain_real, datapr_real = fun_real_generator(batch_size, datain)
    datain_fake, datapr_fake = fun_fake_generator(model_generator, hiddenin)

    datain_comb = np.concatenate((datain_real, datain_fake), axis = 0)
    datapr_comb = np.concatenate((datapr_real, datapr_fake), axis = 0)

    loss_discriminator_comb = model_discriminator.train_on_batch(datain_comb, datapr_comb)

    prob_discriminator_fake = model_discriminator.predict(datain_fake).mean()
    prob_discriminator_real = model_discriminator.predict(datain_real).mean()

    hiddenin, hiddenpr      = fun_latent(batch_size * 2, hidden_size)
    loss_generator          = model_gan.train_on_batch(hiddenin, hiddenpr)

    if i1%10  == 0:
      image    = fun_plot(model_generator.predict(hiddenin)[:,:,:,0])
      img_name = 'img_file_{:05d}_{:05d}.npy'.format(i0,i1)
      np.save(img_name,image)

      clear_output(wait=True)
      plt.figure(figsize = [10,10])
      plt.imshow(image, cmap = 'gray')
      plt.title("Epoch#: {:03.0f}/{} | Batch#: {:04.0f}/{} | D-Loss: {:05.5f} | G-Loss: {:05.5f} | P-Real: {:0.7f} | P-Fake: {:0.7f} ".format(i0+1, epochs, i1, num_batch, loss_discriminator_comb, loss_generator,prob_discriminator_real,prob_discriminator_fake))
      plt.show()

In [ ]:
#@title 8.7.9. Generate some new images
hiddenin,_               = fun_latent(25, hidden_size)
datain_fake, datapr_fake = fun_fake_generator(model_generator, hiddenin)

f, ax = plt.subplots(5, 5, sharex= True, sharey= True, figsize=(10,10))
f.subplots_adjust(hspace=0)
c = 0
for i0 in range(5):
  for i1 in range(5):
    ax[i0,i1].imshow(datain_fake[c,:,:,0], cmap='gray')
    c = c+1